In [1]:
import torch
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple
import torch
import torch.optim as optim

In [2]:
from model import Net
model = torch.load('modelentire.pth')
model.load_state_dict(torch.load('model.pth'))
loss_function = nn.MSELoss()
model.eval()

Net(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
)

In [3]:
df = pd.read_pickle('Dataset')
df

,Sigma,mu,d,th,th0,Decay
19628,-1.271194,-0.747548,0.0,-1.085085,0.0,"[0.19375629692752577, 0.18294351872284478, 0.1..."
2486,-0.522897,0.229992,0.0,-0.732977,0.0,"[0.192993357268207, 0.1820247479384618, 0.1711..."
64883,1.436271,-0.737692,0.0,0.091825,0.0,"[0.1752133606610032, 0.16091191549296152, 0.14..."
10027,1.629299,1.329376,0.0,-1.526841,0.0,"[0.18759459262951242, 0.17560025481240513, 0.1..."
51126,0.423736,-0.777624,0.0,-0.247653,0.0,"[0.17945400597800243, 0.1659367937378173, 0.15..."
...,...,...,...,...,...,...
69892,0.814175,0.781662,0.0,-0.878862,0.0,"[0.1882601086365116, 0.17643418646482018, 0.16..."
72626,-0.020505,-0.181898,0.0,-0.651101,0.0,"[0.18716586487573802, 0.17512817120621807, 0.1..."
17249,0.133407,1.380844,0.0,0.863656,0.0,"[0.1928662308439771, 0.18225947646392704, 0.17..."
38159,1.210575,-0.099357,0.0,1.416446,0.0,"[0.1803007929163741, 0.16759557921691812, 0.15..."


In [4]:
def weighted_mse(u_exp: np.ndarray, u: np.ndarray, time: np.ndarray, eps: float = + 10 ** (-15)) -> Tuple[np.ndarray, np.ndarray]:
    dt = np.diff(time, n=1, axis=0)
    dt = np.append(dt, dt[-1])

    amplitude = np.dot(dt, u * u_exp) / np.dot(dt, u ** 2)
    e = (u_exp - amplitude * u) / (u_exp + eps) * dt

    loss = np.dot(e, e)
#     if (np.log(loss) == math.inf):
#         print('u_exp\n', u_exp)
#         print('u\n', u)
#         print('amplitude = ', amplitude)
#         print ('loss = ', loss)
#         sys.exit()
    return loss, amplitude



t = 1e-3 * np.array([
    0.135, 0.180, 0.235, 0.315, 0.365, 0.420,
    0.485, 0.560, 0.650, 0.750, 0.865, 1.000,
    1.155, 1.335, 1.540, 1.780, 2.055, 2.370,
    2.740, 3.160, 3.650, 4.215, 4.870, 5.625,
    6.495, 7.500, 8.660, 10.00, 11.55, 13.34,
    15.40, 17.79, 20.54, 23.72, 27.39, 31.63,
    36.52, 42.17, 48.70, 56.24, 64.94, 74.99,
    86.60, 100.0, 115.5, 133.4
])

In [5]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

device = get_device()

def appending_to_tensor(tensor, rand_2_strs):
    tmp_tensor = torch.tensor([rand_2_strs.get('Sigma').iloc[0],rand_2_strs.get('Sigma').iloc[1],
                               rand_2_strs.get('mu').iloc[0],rand_2_strs.get('mu').iloc[1],
                               rand_2_strs.get('th').iloc[0],rand_2_strs.get('th').iloc[1],
                               np.log(weighted_mse(np.array(rand_2_strs.get('Decay').iloc[0]),np.array(rand_2_strs.get('Decay').iloc[1]),t)[0])])
    tmp_tensor = torch.unsqueeze(tmp_tensor, 0)
    tmp_tensor = torch.cat((tensor,tmp_tensor),0)
    return(tmp_tensor)

def mean_absolute_percentage_error(y_pred, y_true, eps=10**(-2)): 
    return abs(torch.mean(((y_true-y_pred))/(y_true + eps)))*100


def create_batches_to_device(df, device, batch_size=128):
    tensor = torch.tensor([])
    for i in range (batch_size):
        rand_train_str = df.sample(2)
        tensor = appending_to_tensor(tensor, rand_train_str)
    tensor = tensor.to(device)
    trainset = torch.utils.data.DataLoader(tensor, batch_size=batch_size, shuffle=True)
    return (trainset)
print(device)

cuda:0


In [6]:
net = model.double().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)
df_mape_loss = []
import time
from tqdm import tqdm
batch = 1000
for i in tqdm(range(batch)):
    data_batch = create_batches_to_device(df, device)
    for data in data_batch:
        X = data[:, 0:-1]
        y = data[:,-1]
        output = net(X.double())
        loss = loss_function(output.view(-1), y)
    mape = mean_absolute_percentage_error(output.view(-1), y)
    df_mape_loss.append(mape.item())
    batch = batch + 1

100%|██████████| 1000/1000 [04:09<00:00,  4.01it/s]


In [13]:
print(len(df_mape_loss))
lol = pd.DataFrame(data=df_mape_loss, columns=["Ошибка сети в %"])
lol.to_pickle('Mape_for_hist')

1000


In [14]:
hist_df = pd.read_pickle('Mape_for_hist')
hist_df

,Ошибка сети в %
0,1.549410
1,1.767146
2,2.562353
3,0.271854
4,0.353050
...,...
995,0.989469
996,3.147639
997,2.376084
998,4.937321


In [25]:
fig = px.histogram(hist_df, x="Ошибка сети в %", nbins=10000)
fig.update_xaxes(range=[0, 100])
fig.show()